## This is the notebook for the new coffee.csv file we found.

Cleaning and exploration code below...

In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
import pycountry
import re
from sqlalchemy import create_engine, text, inspect

In [ ]:
# Read the csv into a Pandas DataFrame
coffee_df = pd.read_csv('../Resources/coffee.csv', encoding='utf-8')
coffee_df.head()

In [ ]:
# Get a brief summary of DataFrame
coffee_df.info()

In [ ]:
# Drop unnecessary columns 
coffee_df = coffee_df.drop(columns=['all_text', 'est_price', 'review_date', 'agtron', 'location', 'with_milk', 'desc_3', 'desc_4'])

In [ ]:
# Specify the columns to clean with regex
columns_to_clean = ['roaster','desc_1', 'desc_2']

# Define a regex pattern to match special characters
special_characters_pattern = r'[^a-zA-Z0-9\s,.-é"]'

# Clean chosen columns
for column in columns_to_clean:
    coffee_df[column] = coffee_df[column].str.replace(special_characters_pattern, '', regex=True)

### Work with the 'origin' column

In [ ]:
# Get details about 'origin' column
coffee_df['origin'].value_counts()

In [ ]:
# Remove the dot at the end of string in 'origin' column
coffee_df['origin'] = coffee_df['origin'].str.rstrip('.')
coffee_df['origin'].value_counts()

In [ ]:
# Delete all rows with different iterations of 'not disclosed' from 'origin' column

# Define a regex pattern
pattern = re.compile(r'not\s*disclosed', flags=re.IGNORECASE)

# Apply this to 'origin' column
coffee_df = coffee_df[~coffee_df['origin'].str.contains(pattern, na=False) | coffee_df['origin'].isna()]

In [ ]:
# Delete rows with NaN values from 'origin' column
coffee_df = coffee_df.dropna(subset=['origin'])

In [ ]:
# Create new column with specific 'country of origin' values

# Define the 'origin' column 
old_origin_column = 'origin'

# Create a new column to store the split result
new_origin_column = 'country_of_origin'

# Function to split the last word and add it to the new column
def split_last_word_except_semicolon(text):
    if pd.isna(text):  # Check if the value is NaN
        return ''
    words = re.split(r'\s*;\s*|\s+', text)
    return words[-1]

# Apply the function to create a new column
coffee_df['country_of_origin'] = coffee_df['origin'].apply(split_last_word_except_semicolon)

# Print the updated DataFrame
coffee_df.head()

In [ ]:
# Check values
coffee_df['country_of_origin'].value_counts()

In [ ]:
# Change the datatype of the specified column to string
coffee_df['country_of_origin'] = coffee_df['country_of_origin'].astype(str)

# Set the cutoff value
cutoff = 12

# Create a mask for rows to keep
keep_mask = coffee_df['country_of_origin'].map(coffee_df['country_of_origin'].value_counts()) >= cutoff

# Keep only the rows where the count is at least the cutoff
coffee_df = coffee_df[keep_mask]

In [ ]:
coffee_df.info()

In [ ]:
# Drop the rows with blank values from the original DataFrame
rows_with_blank_values = coffee_df[coffee_df['country_of_origin'].isna() | (coffee_df['country_of_origin'] == '')]

if not rows_with_blank_values.empty:
    coffee_df.drop(rows_with_blank_values.index, inplace=True)
    
# Drop all rows containing 'Africa' as a country of origin
string_to_drop = 'Africa'
coffee_df = coffee_df[~coffee_df['country_of_origin'].str.contains(string_to_drop, case=False, na=False)]

coffee_df.head()

In [ ]:
coffee_df = coffee_df.drop(columns='origin')

In [ ]:
coffee_df.country_of_origin.value_counts().sum()

In [ ]:
coffee_df.info()

## Get latitude and longitude for countries_of_origin

In [ ]:
# Check unique values from the 'country_of_origin' column
coffee_df['country_of_origin'].unique()

In [ ]:
# Change country names to match country names in pycountry library

# Define a dictionary of old and new values
values_to_update = {'America': 'United States',
                    'Hawaii': 'United States', 
                    'Democratic_Republic_of_Congo' : 'Congo, The Democratic Republic of the', 
                    'Salvador': 'El Salvador', 
                    'Costa_Rica': 'Costa Rica', 
                    'Tanzania': 'Tanzania, United Republic of', 
                    'Papua_New_Guinea': 'Papua New Guinea', 
                    'Bolivia': 'Bolivia, Plurinational State of', 
                    'Sumatra': 'Indonesia', 
                    'Taiwan': 'China'
                   }

# Create a mask for rows that need updating
update_mask = coffee_df['country_of_origin'].isin(values_to_update.keys())

# Update the values in the 'country_of_origin' column
coffee_df.loc[update_mask, 'country_of_origin'] = coffee_df.loc[update_mask, 'country_of_origin'].replace(values_to_update)

coffee_df.head()

In [ ]:
# Create a new DataFrame of the 'country_of_origin' column for getting lat and lon
coffee_countries = coffee_df[['country_of_origin']].copy()

# Extract unique values from the 'Category' column
unique_categories = coffee_countries['country_of_origin'].unique()

# Create a new DataFrame with unique values
unique_df = pd.DataFrame({'country_of_origin': unique_categories})

In [ ]:
def get_coordinates(country):
    try:
        country_obj = pycountry.countries.get(name=country)
        geolocator = Nominatim(user_agent="coffee_countries", timeout=20)
        location = geolocator.geocode(country_obj.name)
        return location.latitude, location.longitude
    except AttributeError:
        return None, None

unique_df[['latitude', 'longitude']] = unique_df['country_of_origin'].apply(get_coordinates).apply(pd.Series)

unique_df

In [ ]:
# Merge the DataFrames based on the common column
new_coffee_df = pd.merge(coffee_df, unique_df, on='country_of_origin', how='left')

In [ ]:
new_coffee_df.head()

In [ ]:
new_coffee_df.info()

In [ ]:
# Display the result DataFrame
new_coffee_df.columns

In [ ]:
# Reorder columns in the DataFrame
coffee_df = new_coffee_df[['slug', 'name', 'roaster', 'roast', 'country_of_origin', 'desc_1', 'desc_2', 'latitude', 'longitude', 'rating',
                       'aroma', 'acid', 'body', 'flavor', 'aftertaste',
                       'region_africa_arabia', 'region_caribbean', 'region_central_america', 'region_hawaii', 'region_asia_pacific', 'region_south_america', 
                       'type_espresso', 'type_organic', 'type_fair_trade', 'type_decaffeinated', 'type_pod_capsule', 'type_blend', 'type_estate']]

In [ ]:
coffee_df.info()

In [ ]:
# Drop rows with null values 
coffee_df = coffee_df.dropna()

In [ ]:
coffee_df.info()

In [ ]:
coffee_df.to_csv('../Resources/NEW_coffee_final.csv', index=False)

In [ ]:
engine = create_engine('sqlite:///Data_Engineering.db')

In [ ]:
# Drop the existing table and create a new one with the desired primary key
with engine.connect() as con:
    con.execute(text('''
        CREATE TABLE IF NOT EXISTS coffee_data (
            "slug" VARCHAR, 
            "name" VARCHAR, 
            "roaster" VARCHAR, 
            "roast" VARCHAR, 
            "country_of_origin" VARCHAR, 
            "desc_1" VARCHAR, 
            "desc_2" VARCHAR, 
            "rating" INTEGER,
            "aroma" FLOAT, 
            "acid" FLOAT, 
            "body" FLOAT, 
            "flavor" FLOAT, 
            "aftertaste" FLOAT,
            "region_africa_arabia" INTEGER, 
            "region_caribbean" INTEGER, 
            "region_central_america" INTEGER, 
            "region_hawaii" INTEGER, 
            "region_asia_pacific" INTEGER, 
            "region_south_america" INTEGER, 
            "type_espresso" INTEGER, 
            "type_organic" INTEGER, 
            "type_fair_trade" INTEGER, 
            "type_decaffeinated" INTEGER, 
            "type_pod_capsule" INTEGER, 
            "type_blend" INTEGER, 
            "type_estate" INTEGER,
            PRIMARY KEY ("slug")
        )
    '''))

# Output to the database 
coffee_df.to_sql(name='coffee_data', con=engine, if_exists='replace', index=False)

In [ ]:
# Create a connection
connection = engine.connect()

# Create an Inspector and get the table names
inspector = inspect(engine)
table_names = inspector.get_table_names()

# Print the table names and some sample data
for table_name in table_names:
    print(f"Table: {table_name}")

    # Use text() to create a SQL expression
    query = text(f"SELECT * FROM {table_name} LIMIT 5")
    sample_data = connection.execute(query).fetchall()

    print("Sample Data:")
    for row in sample_data:
        print(row)
    print("\n")

# Close the connection
connection.close()